In [1]:
import bs4
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import csv

In [2]:
#Reading the html data from the url to p_html
my_url = 'https://www.cyber.gov.au/acsc/view-all-content/alerts'
uClient = uReq(my_url)
p_html = uClient.read()
uClient.close()

In [3]:
#Parsing html
parsed_html = soup(p_html, "html.parser")

In [4]:
#Splitting the html into a list of threats
rows = parsed_html.findAll("div", {"class":"views-row"})
#deleting unrequired row from the list
del rows[0]

In [5]:
#Creating .csv file in the data folder and writing the required information from the list to .csv file
with open('data/cyber_gov_au.csv', 'w') as file:
    w = csv.writer(file)
    #write header row
    w.writerow(['Threat_date', 'Title', 'Threat_level', 'Summary', 'Source'])
    for row in rows:
        date = row.find("p", {"class":"acsc-date"}).get_text()
        title = row.find("p", {"class":"acsc-title"}).get_text()
        summary = row.find("p", {"class":"acsc-summary"}).get_text()
        src_link = ("https://www.cyber.gov.au" + row.a['href'])

        uClient = uReq(src_link)
        detailed_page = uClient.read()
        uClient.close()
        dp_parsed = soup(detailed_page, "html.parser")
        Threat = dp_parsed.find("div", {"class":"field field--name-field-alert-status field--type-entity-reference field--label-inline"})
        lvl = Threat.find("div", {"class":"field__item"}).get_text()

        summary = summary.replace(",", ";")
        w.writerow([date, title, lvl, summary, src_link])
